In [1]:
import pyspark
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("spark://de-zoomcamp.us-central1-c.c.shareprompt-412612.internal:7077") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/01 15:15:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/01 15:15:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
print(spark.sparkContext.uiWebUrl)


http://de-zoomcamp.us-central1-c.c.shareprompt-412612.internal:4041


25/03/01 15:15:15 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
df_green = spark.read.parquet("data/raw/green/*/*")

25/03/01 15:15:38 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/03/01 15:15:53 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [ ]:
df_green.show()

In [ ]:

df_green = df_green.withColumn("VendorID", col("VendorID").cast("int"))

In [ ]:
df_green.printSchema()
df_green.show()

In [ ]:
yellow_schema = yellow_schema = types.StructType([
    types.StructField("VendorID", types.StringType(), True),
    types.StructField("tpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("tpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("passenger_count", types.IntegerType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("RatecodeID", types.IntegerType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("payment_type", types.IntegerType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])

In [ ]:
df_yellow = spark.read.parquet("data/raw/yellow/*/*")

In [ ]:
df_yellow.show()

In [ ]:
df_yellow = df_yellow.withColumn("VendorID", col("VendorID").cast("int"))

In [ ]:
df_yellow.columns

In [ ]:
set(df_yellow.columns) & set(df_green.columns)

In [ ]:
df_green=df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')


df_yellow=df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [ ]:
set(df_yellow.columns) & set(df_green.columns)

In [ ]:
columns = []
yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        columns.append(col)
        

In [ ]:
columns

In [ ]:
df_green.select(columns).show()

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df_green_sel =df_green.select(columns) \
    .withColumn('service_type', F.lit("green"))

In [ ]:
df_yellow_sel =df_yellow.select(columns) \
    .withColumn('service_type', F.lit("yellow"))

In [ ]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [ ]:
df_trips_data.groupBy('service_type').count().show()

In [ ]:
df_trips_data.registerTempTable('trips_data')

In [ ]:
df_result = spark.sql("""
SELECT
    -- Revenue grouping 
    PULocationID AS revenue_zone,
    date_trunc("month", "pickup_datetime") AS revenue_month, 

    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,

    -- Additional calculations
    AVG(passenger_count) AS avg_monthly_passenger_count,
    AVG(trip_distance) AS avg_monthly_trip_distance

    FROM
    trips_data
    GROUP BY 1,2,3
""")

In [ ]:
df_result.write.parquet('data/report/revenue/')

In [ ]:
print(spark.sparkContext.uiWebUrl)
